In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import reciprocal
import scikeras
from scikeras.wrappers import KerasRegressor
import keras_tuner as kt
import os

2025-05-26 23:11:14.949346: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 23:11:14.953437: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 23:11:14.962250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-26 23:11:14.976028: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-26 23:11:14.979835: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-26 23:11:14.995467: I tensorflow/core/platform/cpu_feature_gu

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": np.arange(1, 100),
    "model__learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
    "model__optimizer": ['adam', 'sgd', 'nesterov']
}

In [4]:
# Searching the hyperparameter space with scikit-learn

def build_model(n_hidden, n_neurons, optimizer, learning_rate, input_shape=[8]):
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(n_neurons, activation='relu', input_shape=input_shape))
    for i in range(n_hidden - 1):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))

    if optimizer == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    elif optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(loss='mse', optimizer=optimizer, metrics=["mse"])
    return model

In [5]:
# learning_rates = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

# h = []

# for lr in learning_rates:
#     print(f"Training model with learning_rate={lr}")
#     model = build_model(n_hidden=2, n_neurons=30, optimizer='adam', learning_rate=lr)
    
#     history = model.fit(X_train, y_train, epochs=40, validation_data=(X_valid, y_valid), verbose=0)
#     h.append(history)

# plt.figure(figsize=(10,6))
# for i, h_i in enumerate(h):
#     plt.plot(h_i.history['loss'], label=f"Training loss 1e-{i+1}")
# plt.legend()
# plt.show()

In [6]:
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=5, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/home/piotrp/miniconda3/envs/ml/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1748293878.686694  171647 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-26 23:11:18.687062: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 12: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step
[CV] END model__learning_rate=0.00033604138052772926, model__n_hidden=1, model__n_neurons=19, model__optimizer=adam; total time=   6.1s


/home/piotrp/miniconda3/envs/ml/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 19: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END model__learning_rate=0.00033604138052772926, model__n_hidden=1, model__n_neurons=19, model__optimizer=adam; total time=   8.3s


...

/home/piotrp/miniconda3/envs/ml/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 11: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step
[CV] END model__learning_rate=0.0005788369689719182, model__n_hidden=2, model__n_neurons=91, model__optimizer=nesterov; total time=   4.9s


/home/piotrp/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [0.63909457 0.62430763 0.30237671        nan 0.53628003]
  warnings.warn(
/home/piotrp/miniconda3/envs/ml/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 13: early stopping


RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7f11b25da950>], model=<function build_model at 0x7f11834a20c0>),
                   n_iter=5,
                   param_distributions={'model__learning_rate': [0.003447829244999166,
                                                                 0.0003313389859990192,
                                                                 0.0012738871943916684,
                                                                 0.0028359100447301402,
                                                                 0.0012194543077730419,
                                                                 0.005738...
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'model__optimizer': ['adam', 'sgd',
                                                             'nesterov']},
                   verbose=2)

In [7]:
rnd_search_cv.best_params_

{'model__optimizer': 'adam',
 'model__n_neurons': 19,
 'model__n_hidden': 1,
 'model__learning_rate': 0.00033604138052772926}

In [8]:
with open('rnd_search_params.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv.best_params_, f)

with open('rnd_search_scikeras.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv, f)

In [9]:
# Searching the hyperparameter space with Keras Tuner

def build_model_kt(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3)
    n_neurons = hp.Int("n_neurons", min_value=1, max_value=100)
    learning_rate = hp.Float("learning_rate", min_value=3e-4, max_value=3e-2, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam", "nesterov"])

    if optimizer == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    elif optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        
    model = keras.Sequential()
    model.add(keras.layers.Dense(n_neurons, activation='relu', input_shape=[8]))
    for i in range(n_hidden - 1):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])
    
    return model

In [10]:
random_search_tuner = kt.RandomSearch(build_model_kt, 
                                      objective="val_mse", 
                                      max_trials=10, 
                                      overwrite=True, 
                                      directory="my_california_housing", 
                                      project_name="my_rnd_search"
)

In [11]:
root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = tf.keras.callbacks.TensorBoard(root_logdir)

In [12]:
random_search_tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[tb, es])

Trial 10 Complete [00h 00m 07s]
val_mse: 0.4364100694656372

Best val_mse So Far: 0.29902157187461853
Total elapsed time: 00h 01m 20s


In [13]:
# %load_ext tensorboard
# %tensorboard --logdir {root_logdir}

In [14]:
best_hp = random_search_tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp.values

{'n_hidden': 3,
 'n_neurons': 58,
 'learning_rate': 0.001547752040188945,
 'optimizer': 'adam'}

In [15]:
best_model = build_model_kt(best_hp)
best_model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[es])

Epoch 1/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.5629 - mse: 1.5629 - val_loss: 0.4229 - val_mse: 0.4229
Epoch 2/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4090 - mse: 0.4090 - val_loss: 0.3754 - val_mse: 0.3754
Epoch 3/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3655 - mse: 0.3655 - val_loss: 0.3518 - val_mse: 0.3518
Epoch 4/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3333 - mse: 0.3333 - val_loss: 0.3462 - val_mse: 0.3462
Epoch 5/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3150 - mse: 0.3150 - val_loss: 0.3429 - val_mse: 0.3429
Epoch 6/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3054 - mse: 0.3054 - val_loss: 0.3167 - val_mse: 0.3167
Epoch 7/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3072 - mse: 0.3072 - val_loss: 0.3164 - val_mse: 0.3164
Epoch 8/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2949 - mse: 0.2949 - val_loss: 0.3023 - val_mse: 0.3023
Epoch 9/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/

In [16]:
with open('kt_search_params.pkl', 'wb') as f:
    pickle.dump(best_hp.values, f)

best_model.save('kt_best_model.keras')